In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load & Display data

In [ ]:
# Load the datasets
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')


In [ ]:
# Display the first few rows of each dataframe to understand their structure
print("Training Data:")
print(train_df.head())


In [ ]:
print("\nTest Data:")
print(test_df.head())

## View First Image

In [ ]:
import matplotlib.pyplot as plt
# Extract labels and pixel data
y = train_df['label']  # Labels
X = train_df.drop(columns=['label'])  # Pixel values

# Select the first image from the dataset
first_image = X.iloc[0].values  # Extract pixel values of the first image
first_image_reshaped = first_image.reshape(28, 28)  # Reshape to 28x28

# Display the first image
plt.imshow(first_image_reshaped, cmap='gray')
plt.title(f'Label: {y.iloc[0]}')
plt.show()

### display first few image

In [ ]:
# Display the first 5 images in the dataset
for i in range(5):
    image = X.iloc[i].values.reshape(28, 28)  # Reshape each image
    label = y.iloc[i]  # Get the label for the image
    
    plt.subplot(1, 5, i + 1)  # Create subplots
    plt.imshow(image, cmap='gray')
    plt.title(f'Label: {label}')
    plt.axis('off')  # Hide the axes

plt.show()


## Normalization and Splits

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Normalize pixel values
X_normalized = X / 255.0


In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np

# Convert DataFrames to NumPy arrays and normalize
X_train_np = X_train.to_numpy().reshape(-1, 28, 28, 1)
X_val_np = X_val.to_numpy().reshape(-1, 28, 28, 1)

## Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize the data generator
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

# Fit the generator to your training data
datagen.fit(X_train_np)


## Build Model

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf
# Define and compile the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

## Train Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler


# Define a learning rate schedule function
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Create the LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

# Define the callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    filepath='/kaggle/working/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Train the model with callbacks
history = model.fit(X_train_np, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(X_val_np, y_val),
                    callbacks=[early_stopping, model_checkpoint])


In [ ]:
from tensorflow.keras.models import load_model

# Load the best model saved during training
best_model = load_model('/kaggle/working/best_model.keras')

# Optionally, you can compile the model again if you need to continue training
best_model.compile(optimizer='adam',  # or whatever optimizer you were using
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
best_model.summary()

In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = best_model.evaluate(X_val_np, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


In [ ]:
# Normalize test data
X_test = test_df / 255.0

# Reshape test data
X_test_np = X_test.to_numpy().reshape(-1, 28, 28, 1)

# Make predictions
predictions = best_model.predict(X_test_np)
predicted_labels = predictions.argmax(axis=1)  # Get the class with the highest probability


In [ ]:
# Prepare the submission DataFrame
submission = pd.DataFrame({'ImageId': range(1, len(predicted_labels) + 1),
                           'Label': predicted_labels})


In [ ]:
submission

In [ ]:
# Save the submission file
submission.to_csv('/kaggle/working/submission4.csv', index=False)